# ...So
We now have a dictionary containing dataframes of FDA action dates and stock price time series. Lets open it up from dill and begin cutting out our feature space.

In [1]:
import dill
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook

In [2]:
company_dataframes = dill.load(open('Prices_and_FDA_Dates.pkl', 'r'))
company_list = company_dataframes.keys()

In [3]:
len(company_list)

186

Lets also open the index price and normalize our prices to that

In [4]:
closing_index = dill.load(open("close_price_stats_frame.pkl", "r"))

In [5]:
closing_index

,CP_mean,CP_stdv
2000-01-03,27.105697,28.486195
2000-01-04,25.714626,27.195048
2000-01-05,25.839977,27.284835
2000-01-06,26.257429,27.431282
2000-01-07,28.418530,30.458220
2000-01-10,29.682323,31.791633
2000-01-11,29.124288,30.836872
2000-01-12,28.900803,30.421695
2000-01-13,29.900409,32.670777
2000-01-14,30.369091,32.851835


In [6]:
company_dataframes['NEW'].loc[company_dataframes['NEW']['pdufa?'] == True]

,volume,close,high,open,low,pdufa?
2006-10-06,0.0,11.6,12.8,12.8,11.6,True


In [7]:
testdf = company_dataframes['NEW']

In [8]:
testdf.reset_index(inplace = True)

In [9]:
testdf.loc[testdf['pdufa?']]

,index,volume,close,high,open,low,pdufa?
1699,2006-10-06,0.0,11.6,12.8,12.8,11.6,True


In [10]:
testdf.loc[1699]

index     2006-10-06
volume             0
close           11.6
high            12.8
open            12.8
low             11.6
pdufa?          True
Name: 1699, dtype: object

In [11]:
testdf.loc[1699]['close']

11.6

In [12]:
testind = testdf.index[testdf['pdufa?'] == True]

In [13]:
testind

Int64Index([1699], dtype='int64')

In [14]:
testind[0]

1699

So general idea:
1. iterate through dictionary of dataframes
1. in each dataframe, find each row with a `pdufa`
1. collect the (preceding and following) 120 rows of close prices and volumes
1. return those as an array of close price and volume vectors with the company name and pdufa date as metadata
1. something to the effect of `("Ticker", pdufaDate, (120 preceding close prices and volumes), (120 following close prices and volumes))`

In [15]:
company_dataframes['ABBV'].join(closing_index, how='left')

,volume,close,high,open,low,pdufa?,CP_mean,CP_stdv
2013-01-02,13767900.0,35.1200,35.400,34.92,34.100,False,20.154024,30.430870
2013-01-03,16739300.0,34.8300,35.000,35.00,34.160,False,20.209573,30.325838
2013-01-04,21372100.0,34.3908,34.890,34.62,34.250,False,20.326711,30.424589
2013-01-07,17897100.0,34.4600,35.450,34.15,34.150,False,20.410941,30.500001
2013-01-08,17863300.0,33.7100,34.640,34.29,33.360,False,20.510795,30.449752
2013-01-09,18800400.0,33.9000,33.950,33.61,33.610,False,20.738938,30.879858
2013-01-10,15658100.0,34.0000,34.000,33.66,33.330,False,20.862919,31.083608
2013-01-11,11191500.0,33.8500,33.900,33.59,33.350,False,20.805752,30.941736
2013-01-14,11584900.0,34.0900,34.250,34.01,33.800,False,20.785771,30.914987
2013-01-15,13040200.0,34.6000,34.720,33.71,33.710,False,20.861152,30.795951


In [16]:
data = []
for company in tqdm_notebook(company_list):
    df = company_dataframes[company].reset_index()
    pdufa_dates = df.index[df['pdufa?']].tolist()
    if len(pdufa_dates) > 0:
        for date in pdufa_dates:
            pRange = range(date-120, date)
            fRange = range(date, date+121)
            pCloses, pVolumes, fCloses, fVolumes = [], [], [], []
            for i in pRange:
                try:
                    pCloses.append(df.loc[i]['close'])
                    pVolumes.append(df.loc[i]['volume'])
                except:
                    pCloses.append(None)
                    pVolumes.append(None)
            for i in fRange:
                try:
                    fCloses.append(df.loc[i]['close'])
                    fVolumes.append(df.loc[i]['volume'])
                except:
                    fCloses.append(None)
                    fVolumes.append(None)
            data.append((company, df.loc[date]['index'], (pCloses, pVolumes), (fCloses, fVolumes)))

In [17]:
dill.dump(data, open('stock_price_training_slices.pkl', 'w'))

So theres our data points, stored as slices of the stock price/volume histories 120 days prior to an FDA event. `268*120*2 = 64320` data points in total. Time for some signal processing.

I know this could be done far more elegantly, but I need an adequate solution _yesterday_, not a perfect one next week.

# Part Two

Now I've made a closing price index, so lets normalize the prices to that. I'm also shortening the slices from [120 days prior to day of], to [120 days prior to 7 days prior] for the eventual web facing app. 

Obviously we no longer need following slices for plots, as the data has been annotated. 

In [18]:
norm_data = []
for company in tqdm_notebook(company_list):
    df = company_dataframes[company].join(closing_index, how='left').reset_index()
    pdufa_dates = df.index[df['pdufa?']].tolist()
    if len(pdufa_dates) > 0:
        for date in pdufa_dates:
            pRange = range(date-120, date-7)
            pCloses, pVolumes = [], []
            for i in pRange:
                try:
                    close_price = df.loc[i]['close']
                    volume = df.loc[i]['volume']
                    mean_price = df.loc[i]['CP_mean']
                    stdv_price = df.loc[i]['CP_stdv']
                    pCloses.append((close_price-mean_price)/(stdv_price))
                    pVolumes.append(volume)
                except:
                    pCloses.append(None)
                    pVolumes.append(None)
            norm_data.append((company, df.loc[date]['index'], (pCloses, pVolumes)))

That looks normalized to me, lets rejoin the annotations to the data and begin feature extraction. 

In [19]:
scores = [line.split() for line in open("score_sheet_complete.txt", "r").readlines()]

In [20]:
scores[:2]

[['AAAP', '2016-06-01', '1'], ['AAAP', '2016-12-28', '1']]

In [21]:
norm_data_annotated = []
for datum, score in zip(norm_data, scores):
    if datum[0] == score [0] and datum [1] == score[1]:
        norm_data_annotated.append((datum[0], datum[1], 
                                    score[2], datum[2] ))
    else:
        print "whoops theres a mismatch"
        

In [22]:
norm_data_annotated[:2]

[('AAAP',
  '2016-06-01',
  '1',
  ([-0.12174738784081005,
    -0.11649757460863203,
    -0.12202718753675167,
    -0.10701232720700836,
    -0.098575729382038818,
    -0.1192150549485034,
    -0.13214151674067587,
    -0.11797484551305558,
    -0.082288199923163566,
    -0.1143065235110669,
    -0.11929035332875815,
    -0.11892191948170955,
    -0.12568038792527547,
    -0.12710162195805946,
    -0.12395220359326434,
    -0.1227671908381152,
    -0.092712355058081633,
    -0.083297778426778893,
    -0.082662390476770778,
    -0.090411169273986777,
    -0.066076615447791726,
    -0.090436371473833085,
    -0.086162818845528177,
    -0.13408592795608898,
    -0.091785042591575619,
    -0.12446081332632397,
    -0.10579387547911373,
    -0.084261111022572513,
    -0.075454238566828313,
    -0.072115745708099421,
    -0.066044114655135239,
    -0.11694274320792392,
    -0.10622732599259678,
    -0.091626651628569672,
    -0.094566589786288124,
    -0.09096207655742361,
    -0.08623685577

In [23]:
dill.dump(norm_data_annotated, open('normalized_stock_price_slices.pkl', 'w'))

That looks normalized and serialized to me. Time to run some peak detection and get creative with feature extraction. 